In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%capture
!cp -r /content/drive/MyDrive/esmini.zip /content/esmini.zip
!unzip /content/esmini.zip

In [4]:
%%capture
!apt-get install -y xvfb x11-utils
!pip install ultralytics

In [9]:
import sys, os
!wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/xvfb -O ../xvfb
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'
!cp -r /content/drive/MyDrive/xvfb /content/xvfb
!chmod 755 /content/xvfb

In [ ]:
%rm -r sim_loop

rm: cannot remove 'sim_loop': No such file or directory


In [10]:
!/content/xvfb start
%cd /content/esmini
!chmod 755 bin/esmini
%cd /content
!git clone https://github.com/patrickw16/sim_loop.git

Starting virtual X frame buffer: Xvfb.
/content/esmini
/content
Cloning into 'sim_loop'...
remote: Enumerating objects: 3835, done.
remote: Counting objects: 100% (787/787), done.
remote: Compressing objects: 100% (270/270), done.
remote: Total 3835 (delta 620), reused 643 (delta 516), pack-reused 3048 (from 1)
Receiving objects: 100% (3835/3835), 90.95 MiB | 34.87 MiB/s, done.
Resolving deltas: 100% (3269/3269), done.
Updating files: 100% (3408/3408), done.


In [11]:
!git clone https://...@github.com/patrickw16/odd_coverage.git
%cd odd_coverage
!git checkout ThreeDimensional
%cd /content

Cloning into 'odd_coverage'...
remote: Enumerating objects: 2809, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 2809 (delta 26), reused 28 (delta 11), pack-reused 2758 (from 1)
Receiving objects: 100% (2809/2809), 44.42 MiB | 31.85 MiB/s, done.
Resolving deltas: 100% (2232/2232), done.
Updating files: 100% (4838/4838), done.
/content/odd_coverage
Branch 'ThreeDimensional' set up to track remote branch 'ThreeDimensional' from 'origin'.
Switched to a new branch 'ThreeDimensional'
/content


In [12]:
%%capture
!sudo apt install texlive texlive-latex-extra texlive-fonts-recommended dvipng
!pip install latex
!sudo apt install cm-super

In [13]:
import numpy as np
import subprocess
import sys
import os.path
import shutil
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from IPython.core.display import SVG
from pathlib import Path
from multiprocessing.pool import ThreadPool

from sim_loop.evaluation.cube_based_evaluation import CubeBasedEvaluation, fail_ratio
from sim_loop.evaluation.variance_bounded_testing_method.simulation.parameter_set_generation import modify_xml
from odd_coverage.odd_coverage.coverage_param_lvl.coverage_param_lvl import CoverageParamLvl

# Generate variance bounded test cases
def generate_variance_bounded_test_cases(prior_path, number_of_test_values):
  my_coverage_param = CoverageParamLvl(
                      number_of_test_values=number_of_test_values,
                      trace_epsilon=0.0075,
                      epsilon_buffer=0.005,
                      number_of_attempts=1,
                      combined_points_data_path=prior_path)

  param_values, cov_contribution, updated_cluster_traces, adapted_kmeans_labels, re_sampled_scaled, scaler, weighted_within_variance = my_coverage_param.get_optimised_values()
  points = scaler.inverse_transform(re_sampled_scaled)

  #SVG(filename='/content/var_poc_3d_example.svg')
  return points, adapted_kmeans_labels, param_values


# Generate scenarios
def generate_scenarios(param_values):
  for idx, parameter in enumerate(param_values):

      new_s_delta = param_values[idx, 0]  # New value for s_delta
      new_v_delta = param_values[idx, 1]  # New value for v_delta
      new_a_dec = param_values[idx, 2]  # New value for a_dec

      input_file = '/content/sim_loop/scenarios/cut-in.xosc'  # Replace with your input file path
      output_file = f'/content/sim_loop/scenarios/variance_bounded/{idx}_{new_a_dec}_cut-in.xosc'  # Replace with your desired output file path

      modify_xml(input_file, output_file, new_s_delta, new_v_delta)
  return None


def create_folder(path):
  # Specify the name of the new folder
  folder_name = Path(path)
  # Create the new folder
  try:
      folder_name.mkdir(parents=True, exist_ok=True)
      print(f"Folder '{folder_name}' created successfully.")
  except Exception as e:
      print(f"An error occurred: {e}")


def launch_scenario(index):
    p = subprocess.run(
        ['python'] + ['/content/sim_loop/evaluation/variance_bounded_testing_method/simulation/variance_bounded_colab_execution.py'] + ['--window'] + [str(60)] + [str(60)] + [str(800)] + [str(400)] + ['--headless'] + ['--osc'] + [f'/content/sim_loop/scenarios/variance_bounded/{str(index)}_{str(param_values[index, 2])}_cut-in.xosc'] + ['--logfile_path'] + [f'/content/{str(index)}_log.txt'],
    )

def simulation_run(param_values):
    n_runs = len(param_values)
    print('Launching {} scenarios'.format(n_runs))

    with ThreadPool() as p:
        p. map(launch_scenario, range(n_runs))

    print()

def delete_log_txt(path):
  # Specify the folder to search in
  folder_path = Path(path)
  file_name = "log.txt"
  file_path = folder_path / file_name

  # Check if the file exists and delete it
  if file_path.is_file():
      try:
          file_path.unlink()
          print(f"File '{file_name}' deleted successfully.")
      except Exception as e:
          print(f"An error occurred while deleting the file: {e}")
  else:
      print(f"File '{file_name}' not found in '{folder_path}'.")


#move all log files to logs folder
def move_log_files():
  # Move all log files to the logs folder
  log_files = glob.glob("*.txt")  # Find all files ending with .log in the current directory
  for log_file in log_files:
    try:
      shutil.move(log_file, "logs/")  # Move the file to the logs/ directory
    except FileNotFoundError:
      print(f"Error: The 'logs' directory does not exist.")
    except shutil.Error as e:
      print(f"Error moving file {log_file}: {e}")


#zip the logs folder
def zip_logs():
  # Check if the logs directory exists
  if os.path.exists("logs"):
    try:
      shutil.make_archive("logs", "zip", "logs")
      print("Logs folder zipped successfully to logs.zip")
    except Exception as e:
      print(f"Error zipping logs folder: {e}")
  else:
    print("Error: logs folder not found.")


def remove_tree(path):
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(path)


def compile_log_data():
  log_file = "logs"
  log_directory = os.path.join(os.path.expanduser('~'), f"/content/{log_file}")

  # Get a sorted list of all .txt files in the directory
  log_files = sorted([f for f in os.listdir(log_directory) if f.endswith('.txt')])

  # Initialize a list to hold all parameter dictionaries
  all_parameters = []

  # Loop through each log file and read its contents
  for filename in log_files:
      log_file_path = os.path.join(log_directory, filename)

      # Open the log file and read its contents
      with open(log_file_path, 'r') as file:
          log_contents = file.readlines()  # Read lines into a list

      # Initialize a dictionary to hold parameters
      parameters = {}
      parameters_started = False
      collision_detected = False  # Flag to check for collision line

      # Loop through the lines to find "6 parameters:" and extract parameters
      for line in log_contents:
          if "6 parameters:" in line:
              parameters_started = True  # Start reading parameters
              continue  # Skip the line with "6 parameters:"

          # Check for the collision line
          if "Collision" in line:
              collision_detected = True

          if parameters_started:
              # Split the line into key and value
              if '=' in line:
                  key, value = line.split('=')
                  key = key.replace(' ', '')
                  value = value.replace(' ', '')
                  value = value.replace("\n", "")
                  parameters[key.strip()] = float(value.strip())  # Store as float
                  if key == "EgoStartS":
                      parameters_started = False

      # Add collision information to the parameters dictionary
      parameters['CollisionDetected'] = collision_detected
      parameters['Filename'] = filename  # Optionally include the filename

      # Append the parameters dictionary to the list
      all_parameters.append(parameters)

  # Create a DataFrame from the list of dictionaries
  df = pd.DataFrame(all_parameters)

  # Save the DataFrame to a CSV file
  output_file = f'{log_directory}/compiled_log_data.csv'
  df.to_csv(output_file, index=False)

  print(f"Data saved to {output_file}")


def evaluate_metric(using_kmeans_labels=False, points=None, param_values=None, adapted_kmeans_labels=None, a=None, scale=None, loc=None):
  # Order the sim log to align the order of the simulation runs with the param values (test case values)
  df_sim = pd.read_csv('/content/logs/compiled_log_data.csv')
  df_sim['FileNumber'] = df_sim['Filename'].str.extract(r'(\d+)').astype(int) # Extract the first value from the Filename
  result = df_sim[['FileNumber', 'CollisionDetected']]
  ordered_sim_logs = result.sort_values(by='FileNumber')
  ordered_sim_logs.reset_index(drop=True, inplace=True)
  pass_fail_logs = ordered_sim_logs['CollisionDetected'].to_numpy()

  if using_kmeans_labels:
    points = points
    groups = adapted_kmeans_labels.astype(int)
    param_values = param_values

    pass_fail_points = np.zeros(len(groups))
    for i in range(len(groups)):
        group = groups[i]
        pass_fail_points[i] = pass_fail_logs[group]

    pass_fail = pass_fail_points.astype(bool)
  else:
    #simple alternative
    points = param_values
    pass_fail = pass_fail_logs
    groups = np.arange(len(param_values))

  #Compare
  comparison_gt_evaluation = CubeBasedEvaluation(gt_cube_sizes=gt_cubes_sizes,
                                              gt_sub_cube_centers=gt_sub_cube_centers,
                                              gt_pass_fail=gt_pass_fail)
  sub_cubes_with_points = comparison_gt_evaluation.evaluate_gt_cubes_based_on_eval_points(points=points,
                                                                                          groups=groups,
                                                                                          pass_fail=pass_fail)

  sub_cubes_with_points = comparison_gt_evaluation.assign_all_sub_cubes(sub_cubes_with_points=sub_cubes_with_points,
                                                                      points=points,
                                                                      groups=groups,
                                                                      pass_fail=pass_fail)

  #comparison_gt_evaluation.plot_cubes(sub_cubes_with_points=sub_cubes_with_points,
  #                                  points=points)

  metric = comparison_gt_evaluation.calculate_comparison_metric(sub_cubes_with_points=sub_cubes_with_points)
  gt_fail_ratio = fail_ratio(gt_pass_fail)
  eval_fail_ratio = fail_ratio(np.array(pass_fail))
  fail_ratio_difference = (fail_ratio(gt_pass_fail)-fail_ratio(np.array(pass_fail)))*100
  abs_fail_ratio_difference = np.abs(fail_ratio_difference)
  number_of_test_cases = len(param_values[:,0])

  eval_dict = {
      'metric': metric,
      'fail_ratio_difference': fail_ratio_difference,
      'absolute_fail_ratio_difference': abs_fail_ratio_difference,
      'number_of_test_cases': number_of_test_cases,
      'gt_fail_ratio': gt_fail_ratio,
      'eval_fail_ratio': eval_fail_ratio,
      'points': points,
      'pass_fail': pass_fail,
      'a': a,
      'scale': scale,
      'loc': loc,
  }
  return eval_dict


#Load GT data:
gt_cubes_sizes = np.load('/content/sim_loop/evaluation/gt_analysis/data/gt_cube_sizes.npy')
gt_pass_fail = np.load('/content/sim_loop/evaluation/gt_analysis/data/gt_pass_fail.npy')
gt_sub_cube_centers = np.load('/content/sim_loop/evaluation/gt_analysis/data/gt_sub_cube_centers.npy')

In [ ]:
points, adapted_kmeans_labels, param_values = generate_variance_bounded_test_cases(
    './sim_loop/evaluation/variance_bounded_testing_method/data/prior/prior_points_s_delta_v_delta_ego_max_dec_-3_10_0.npy',2)

I will break since no more clusters in red zone.
I successfully finished. There are 1 successful attempts.


In [ ]:
create_folder(path='/content/sim_loop/scenarios/variance_bounded')
generate_scenarios(param_values)

Folder '/content/sim_loop/scenarios/variance_bounded' created successfully.


In [ ]:
simulation_run(param_values=param_values)

Launching 2 scenarios


KeyboardInterrupt: 

In [ ]:
create_folder(path='/content/logs')
delete_log_txt(path='/content')
move_log_files()
compile_log_data()

Folder '/content/logs' created successfully.
File 'log.txt' deleted successfully.
Data saved to /content/logs/compiled_log_data.csv


In [ ]:
eval_dict = evaluate_metric(param_values=param_values, points=points, adapted_kmeans_labels=adapted_kmeans_labels)

Sub-cube 1035 contains points from group [np.int64(0)] (original cube center: [44. 32.  5.]): [array([44.51143785, 32.03614069,  4.92040898])]
Sub-cube 835 contains points from group [np.int64(1)] (original cube center: [22. 22.  5.]): [array([22.37424691, 21.25803745,  4.87674222])]
after update -------------
Sub-cube 1035 contains points from group [np.int64(0)] (original cube center: [44. 32.  5.]): [array([44.51143785, 32.03614069,  4.92040898])]
Sub-cube 835 contains points from group [np.int64(1)] (original cube center: [22. 22.  5.]): [array([22.37424691, 21.25803745,  4.87674222])]
Number of unique sub-cubes: 2
Sub-cube 835 already has points assigned
Sub-cube 1035 already has points assigned
Number of unique sub-cubes: 2080
Number of total sub cubes: 2080


In [ ]:
eval_dict

{'metric': 63.02884615384615,
 'fail_ratio_difference': 36.97115384615385,
 'absolute_fail_ratio_difference': np.float64(36.97115384615385),
 'number_of_test_cases': 2,
 'gt_fail_ratio': 0.3697115384615385,
 'eval_fail_ratio': 0,
 'points': array([[44.51143785, 32.03614069,  4.92040898],
        [22.37424691, 21.25803745,  4.87674222]]),
 'pass_fail': array([ True,  True]),
 'a': None,
 'scale': None,
 'loc': None}

In [ ]:
remove_tree(path='/content/logs')
remove_tree(path='/content/sim_loop/scenarios/variance_bounded')

In [ ]:
# for looping this (or even parallelize it?)
    # decide which prior to use
    # generate test case with variance bounded method
    # generate folder for scenarios
    # generate scenarios (based on test cases)
    # simulate test cases
    # create folder for logs
    # move logs to folder
    # create csv out of logs
    # calculate metric
    # store in dictionary - additionally using the prior paramters
    # remove folder with logs
    # remove folder with scenarios

In [17]:
import itertools
import json

# Prior: Parameters and respective arrays
array1 = [3, 0, -3, -4] #a
array2 = [10,15] #scale
array3 = [-5,0,5] #loc

# Generate all possible combinations
combinations = list(itertools.product(array1, array2, array3))

amount_of_test_cases = 30
idx_stop = 20
idx_start = 11
evaluation_dict = dict()

# loop over the combinations
for idx, combo in enumerate(combinations):
    if idx_start > idx:
      print(f'idx = {idx}, jumping over')
      continue
    else:
      if idx > idx_stop:
        continue
      else:
        print(f'idx = {idx}, starting')
        a = combo[0]
        scale = combo[1]
        loc = combo[2]

        points, adapted_kmeans_labels, param_values = generate_variance_bounded_test_cases(
        f'./sim_loop/evaluation/variance_bounded_testing_method/data/prior/prior_points_s_delta_v_delta_ego_max_dec_{a}_{scale}_{loc}.npy',amount_of_test_cases)

        create_folder(path='/content/sim_loop/scenarios/variance_bounded')
        generate_scenarios(param_values)

        simulation_run(param_values=param_values)

        create_folder(path='/content/logs')
        delete_log_txt(path='/content')
        move_log_files()
        compile_log_data()

        eval_dict = evaluate_metric(param_values=param_values, points=points, adapted_kmeans_labels=adapted_kmeans_labels, a=a, scale=scale, loc=loc)
        evaluation_dict[idx] = eval_dict

        remove_tree(path='/content/logs')
        remove_tree(path='/content/sim_loop/scenarios/variance_bounded')


# Save
df = pd.DataFrame.from_dict(evaluation_dict, orient='index')
df.to_csv(f'evaluation_dict_{amount_of_test_cases}_till_{idx_stop}_of_{len(combinations)}.csv', index=False)

idx = 0, jumping over
idx = 1, jumping over
idx = 2, jumping over
idx = 3, jumping over
idx = 4, jumping over
idx = 5, jumping over
idx = 6, jumping over
idx = 7, jumping over
idx = 8, jumping over
idx = 9, jumping over
idx = 10, jumping over
idx = 11, starting
I adjusted the rate to:0.0033333333333333335
I adjusted the rate to:0.0044444444444444444
I adjusted the rate to:0.005925925925925926
1 8.36607795917764e-05
I adjusted the rate to:0.007901234567901235
2 8.212425461920005e-05
3 8.156705622816382e-05
I adjusted the rate to:0.010534979423868314
4 7.505108958833024e-05
5 7.400500615324722e-05
I adjusted the rate to:0.014046639231824419
6 5.8179797265022094e-05
I will break since no more clusters in red zone.
I successfully finished. There are 1 successful attempts.
Folder '/content/sim_loop/scenarios/variance_bounded' created successfully.
Launching 30 scenarios

Folder '/content/logs' created successfully.
File 'log.txt' deleted successfully.
Data saved to /content/logs/compiled_lo